# Training the CNN for temperature prediction


In [1]:

import os
from CNN import SequenceCNN
from torch.utils.data import DataLoader as TorchDataLoader

In [2]:
import torch
from torch.optim import lr_scheduler

import torch.optim as optim
import torch.nn as nn
from PrecipitationDataset import PrecipitationDataset
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms

In [3]:
dataset = PrecipitationDataset('data_prcp_summed')

100%|██████████| 562/562 [00:05<00:00, 96.24it/s] 


In [ ]:
import optuna
from tqdm import tqdm

def objective(trial):
    # Remove activations_map from here, since it is already defined and used in cell 8.
    # Also, do not redefine batch_size or train_loader, as they are already defined globally.
    # You can remove this placeholder entirely.

    # Use strings to represent tuples for Optuna compatibility
    hidden_dims_options = ["32,32", "32,64", "64,64", "64,32", "32,128", "128,32","64,128","128,64", "128,128", "64,128", "32,256", "128,256", "256,256"]
    kernel_sizes_options = ["3,3", "5,3", "3,5", "5,5", "5,7", "7,5", "7,7", "9,5", "5,9", "9,9", "11,5", "5,11", "11,11", "13,5", "5,13", "13,13"]
    # Define activations_options outside the objective function to avoid dynamic value space error
    activations_options = ["relu,relu", "tanh,relu", "leaky_relu,sigmoid", "sigmoid,tanh", "relu,sigmoid", "tanh,tanh", "leaky_relu,leaky_relu", "tanh,leaky_relu", "relu,leaky_relu", "sigmoid,sigmoid", "sigmoid,leaky_relu", "relu,tanh", "tanh,relu", "leaky_relu,relu", "selu,leaky_relu"]
    hidden_dims_str = trial.suggest_categorical("hidden_dims", hidden_dims_options)
    kernel_sizes_str = trial.suggest_categorical("kernel_sizes", kernel_sizes_options)
    activations_str = trial.suggest_categorical("activations", activations_options)

    hidden_dims = tuple(map(int, hidden_dims_str.split(',')))
    kernel_sizes = tuple(map(int, kernel_sizes_str.split(',')))
    activations = tuple(activations_str.split(','))

    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", (16, 32, 64))
    num_epochs = 5
    sample_input, sample_target = dataset[0]
    input_dim = sample_input.shape[-1]
    output_dim = sample_target.shape[-1]


    model = SequenceCNN(
        input_dim=input_dim,
        hidden_dims=list(hidden_dims),
        kernel_sizes=list(kernel_sizes),
        output_dim=output_dim,
        activations=activations
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    train_loader = TorchDataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        print(f"Starting training epoch {epoch + 1}/{num_epochs}")
        print(f"Learning rate: {lr}, Batch size: {batch_size}, Hidden dims: {hidden_dims}, Kernel sizes: {kernel_sizes}, Activations: {activations}")
        model.train()
        for batch in tqdm(train_loader):
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    return loss.item()

# Example usage:
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=10)

c:\Users\henri\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each set
batch_size = 32  # You can adjust this as needed

train_loader = TorchDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = TorchDataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = TorchDataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("Using CPU")

CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3060 Laptop GPU


In [8]:
import optuna
import warnings
warnings.filterwarnings("ignore")
# Run Optuna hyperparameter search
study = optuna.create_study(
    study_name="CNN_Sequence_Rainfall_Prediction_v2",
    storage="sqlite:///optuna_study.db",
    load_if_exists=True,
    direction="minimize",
)

study.optimize(objective, n_trials=20, show_progress_bar=True)

# Get the best config
best_params = study.best_params
print(f"Best params: {best_params}")


[I 2025-06-08 20:16:12,688] A new study created in RDB with name: CNN_Sequence_Rainfall_Prediction_v2
  0%|          | 0/20 [00:00<?, ?it/s]

Starting training epoch 1/5
Learning rate: 0.004843035170136968, Batch size: 64, Hidden dims: (128, 256), Kernel sizes: (5, 11), Activations: ('leaky_relu', 'leaky_relu')


  0%|          | 0/20 [01:40<?, ?it/s]


[W 2025-06-08 20:17:53,310] Trial 0 failed with parameters: {'hidden_dims': '128,256', 'kernel_sizes': '5,11', 'activations': 'leaky_relu,leaky_relu', 'lr': 0.004843035170136968, 'batch_size': 64} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\henri\AppData\Local\Programs\Python\Python313\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\henri\AppData\Local\Temp\ipykernel_33472\3757253016.py", line 57, in objective
    optimizer.step()
    ~~~~~~~~~~~~~~^^
  File "c:\Users\henri\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\optim\optimizer.py", line 493, in wrapper
    out = func(*args, **kwargs)
  File "c:\Users\henri\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\optim\optimizer.py", line 91, in _use_grad
    ret = func(self, *args, **kwargs)
  File "c:\Users\henri\AppData\Local\Programs\Python\Python313\Lib\site-packages\

KeyboardInterrupt: 

In [ ]:
activations_map = {
    'relu': torch.nn.ReLU(),
    'tanh': torch.nn.Tanh(),
    'sigmoid': torch.nn.Sigmoid(),
    'leaky_relu': torch.nn.LeakyReLU(),
    'elu': torch.nn.ELU(),
    'gelu': torch.nn.GELU(),
    'selu': torch.nn.SELU(),
    'none': nn.Identity()
}
# Prepare activations
activations = [activations_map[act] for act in best_params['activations'].split(',')]

# Build and train the best model on the full training set
sample_input, sample_target = dataset[0]
input_dim = sample_input.shape[-1]
output_dim = sample_target.shape[-1]
best_model = SequenceCNN(
    input_dim=input_dim,
    hidden_dims=list(best_params['hidden_dims']),
    kernel_sizes=list(best_params['kernel_sizes']),
    output_dim=output_dim,
    activations=activations
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)

optimizer = optim.Adam(best_model.parameters(), lr=best_params['lr'])
criterion = nn.MSELoss()

# Use the full train_loader for training
num_epochs = 5
for epoch in range(num_epochs):
    best_model.train()
    for batch in train_loader:
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = best_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluate on the test set
best_model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = best_model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test loss: {test_loss}")


AssertionError: hidden_dims and kernel_sizes must have the same length

In [ ]:
# Save the best model
model_path = Path("best_cnn_model.pth")
torch.save(best_model.state_dict(), model_path)
print(f"Best model saved to {model_path}")
